# 교정 RAG 프로세스: 동적 수정 기능을 갖춘 Retrieval-Augmented Generation

## 개요

교정 RAG(Retrieval-Augmented Generation) 프로세스는 정보 검색과 응답 생성을 더욱 발전시킨 시스템입니다. 표준 RAG 방식을 확장하여 **검색 과정을 동적으로 평가하고 수정**합니다. 이 시스템은 벡터 데이터베이스, 웹 검색, 언어 모델을 결합하여 사용자의 질문에 대해 **정확하고 컨텍스트에 맞는 응답**을 제공합니다.

## 동기

기존의 RAG 시스템은 정보 검색과 응답 생성에서 진전을 이루었으나, 여전히 **관련 없는 정보나 오래된 정보**를 반환하는 경우가 있습니다. 교정 RAG 프로세스는 이러한 문제를 해결하기 위해 설계되었으며 다음과 같은 기능을 제공합니다:

1. 기존 지식 베이스 활용
2. 검색된 정보의 관련성 평가
3. 필요시 웹 검색을 통해 최신 정보 탐색
4. 여러 출처에서 지식 정제 및 결합
5. 가장 적합한 지식을 바탕으로 사람 같은 응답 생성

## 주요 구성 요소

1. **FAISS 인덱스**: 기존 지식 베이스의 효율적인 유사성 검색을 위한 벡터 데이터베이스
2. **검색 평가기**: 검색된 문서의 쿼리 관련성을 평가
3. **지식 정제**: 필요한 경우 문서에서 주요 정보를 추출
4. **웹 검색 쿼리 재작성기**: 로컬 지식이 부족할 경우 웹 검색 쿼리를 최적화
5. **응답 생성기**: 수집된 지식을 바탕으로 사람 같은 응답 생성

## 방법 설명

1. **문서 검색**:
   - FAISS 인덱스를 사용하여 유사성 검색으로 관련 문서를 검색합니다.
   - 상위 k개의 문서를 검색(기본 k=3).

2. **문서 평가**:
   - 각 문서에 대해 관련성 점수를 계산합니다.
   - 최고 관련성 점수를 기준으로 적합한 조치를 결정합니다.

3. **교정 지식 획득**:
   - 높은 관련성(점수 > 0.7): 가장 관련성 높은 문서를 그대로 사용.
   - 낮은 관련성(점수 < 0.3): 쿼리를 수정하여 웹 검색 수행.
   - 중간 관련성(0.3 ≤ 점수 ≤ 0.7): 가장 관련성 높은 문서와 웹 검색 결과를 결합하여 사용.

4. **적응형 지식 처리**:
   - 웹 검색 결과: 주요 포인트만 추출하여 지식 정제.
   - 중간 관련성: 원본 문서와 정제된 웹 검색 결과를 결합하여 사용.

5. **응답 생성**:
   - 언어 모델을 사용해 쿼리와 획득된 지식에 기반한 사람 같은 응답을 생성.
   - 응답에 출처 정보를 포함하여 투명성을 보장.

## 교정 RAG 접근의 이점

1. **동적 수정**: 검색 정보의 품질에 맞춰 관련성과 정확성을 보장.
2. **유연성**: 필요시 기존 지식과 웹 검색을 모두 활용.
3. **정확성**: 정보를 사용하기 전에 관련성을 평가하여 고품질 응답 보장.
4. **투명성**: 출처 정보를 제공하여 정보의 출처를 사용자에게 공개.
5. **효율성**: 대규모 지식 베이스에서 빠르게 검색할 수 있는 벡터 검색 사용.
6. **컨텍스트 이해**: 필요한 경우 여러 출처의 정보를 결합하여 포괄적인 응답 제공.
7. **최신 정보 제공**: 오래된 로컬 지식을 대체하거나 보충할 수 있도록 웹에서 최신 정보 활용.

## 결론

교정 RAG 프로세스는 기존 RAG 접근 방식에서 한 단계 진화한 형태로, 검색 과정을 지능적으로 평가하고 수정하여 RAG 시스템의 한계를 극복합니다. 이 동적 접근 방식은 로컬 지식 베이스와 웹에서 가져온 가장 관련성 높고 최신 정보를 바탕으로 응답을 생성하여 **높은 정확도와 최신 정보가 필요한 응용 분야**에 적합합니다. 연구 지원, 동적 지식 베이스, 고급 질문-응답 시스템에서 특히 유용할 수 있습니다.


<div style="text-align: center;">

<img src="../images/crag.svg" alt="Corrective RAG" style="width:80%; height:auto;">
</div>

### Import relevant libraries

In [1]:
import os
import sys
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field


sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..'))) # Add the parent directory to the path sicnce we work with notebooks
from helper_functions import *
from evaluation.evalute_rag import *

# Load environment variables from a .env file
load_dotenv()

# Set the OpenAI API key environment variable
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
from langchain.tools import DuckDuckGoSearchResults


/opt/anaconda3/envs/kmy_env/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/anaconda3/envs/kmy_env/lib/python3.12/site-packages/deepeval/__init__.py:49: UserWarning: You are using deepeval version 1.4.5, however version 1.4.7 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


### Define files path

In [2]:
path = "../data/Understanding_Climate_Change.pdf"

### Create a vector store

In [4]:
vectorstore = encode_pdf(path)
vectorstore

### Initialize OpenAI language model


In [5]:
llm = ChatOpenAI(model="gpt-4o-mini", max_tokens=1000, temperature=0)

### Initialize search tool

In [8]:
search = DuckDuckGoSearchResults()

### Define retrieval evaluator, knowledge refinement and query rewriter llm chains

In [9]:
from typing import List
from pydantic import BaseModel, Field
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

# 검색 평가기 입력 모델 정의
class RetrievalEvaluatorInput(BaseModel):
    # 문서의 쿼리에 대한 관련성 점수를 0에서 1 사이로 받음
    relevance_score: float = Field(..., description="The relevance score of the document to the query. The score should be between 0 and 1.")

# 쿼리와 문서 관련성 점수 출력 함수 
def retrieval_evaluator(query: str, document: str) -> float:
    # 검색 결과에 대한 관련성 평가 프롬프트 생성
    # 쿼리와 문서가 들어갔을 때 관련성 점수 매기는 프롬프트 
    prompt = PromptTemplate(
        input_variables=["query", "document"],
        template="On a scale from 0 to 1, how relevant is the following document to the query? Query: {query}\nDocument: {document}\nRelevance score:"
    )
    # 프롬프트와 언어 모델을 연결하여 체인 생성, LLM의 출력은 RetrievalEvaluatorInput 구조에 맞춤
    chain = prompt | llm.with_structured_output(RetrievalEvaluatorInput)
    input_variables = {"query": query, "document": document}
    result = chain.invoke(input_variables).relevance_score
    return result


# 지식 정제 입력 모델 정의
class KnowledgeRefinementInput(BaseModel):
    # 문서에서 주요 정보를 추출한 항목 리스트를 담음
    key_points: str = Field(..., description="The document to extract key information from.")

# 문서를 넣었을 때 문서의 핵심 정보를 리스트로 추출함. 
def knowledge_refinement(document: str) -> List[str]:
    # 문서에서 핵심 정보를 추출하는 프롬프트 생성
    # 문서를 넣었을 때 핵심 정보를 추출함. 
    prompt = PromptTemplate(
        input_variables=["document"],
        template="Extract the key information from the following document in bullet points:\n{document}\nKey points:"
    )
    # 프롬프트와 언어 모델을 연결하여 체인 생성, LLM의 출력은 KnowledgeRefinementInput 구조에 맞춤
    chain = prompt | llm.with_structured_output(KnowledgeRefinementInput)
    input_variables = {"document": document}
    result = chain.invoke(input_variables).key_points
    return [point.strip() for point in result.split('\n') if point.strip()]


# 웹 검색용 쿼리 재작성 입력 모델 정의
class QueryRewriterInput(BaseModel):
    # 웹 검색에 적합한 형태로 수정된 쿼리
    query: str = Field(..., description="The query to rewrite.")

# 쿼리를 넣었을 때 웹 검색에 적합하도록 쿼리를 재작성하는 프롬프트 
def rewrite_query(query: str) -> str:
    # 웹 검색에 적합하도록 쿼리를 재작성하는 프롬프트 생성
    prompt = PromptTemplate(
        input_variables=["query"],
        template="Rewrite the following query to make it more suitable for a web search:\n{query}\nRewritten query:"
    )
    # 프롬프트와 언어 모델을 연결하여 체인 생성, LLM의 출력은 QueryRewriterInput 구조에 맞춤
    chain = prompt | llm.with_structured_output(QueryRewriterInput)
    input_variables = {"query": query}
    return chain.invoke(input_variables).query.strip()


### 검색 결과 반환 


In [22]:
# 검색 결과 넣으면 json형식으로, 검색 결과의 제목과 링크를 튜플로 반환 
def parse_search_results(results_string: str) -> List[Tuple[str, str]]:
    """
    검색 결과를 JSON 문자열로 받아, 각 검색 결과의 제목과 링크를 튜플로 반환하는 함수입니다.

    Args:
        results_string (str): JSON 형식의 검색 결과 문자열입니다.

    Returns:
        List[Tuple[str, str]]: 검색 결과의 제목과 링크를 포함하는 튜플의 리스트를 반환합니다.
                               만약 JSON 파싱에 실패할 경우, 빈 리스트를 반환합니다.
    """
    try:
        # Attempt to parse the JSON string
        results = json.loads(results_string)
        # Extract and return the title and link from each result
        return [(result.get('title', 'Untitled'), result.get('link', '')) for result in results]
    except json.JSONDecodeError:
        # Handle JSON decoding errors by returning an empty list
        print("Error parsing search results. Returning empty list.")
        return []

### Define sub functions for the CRAG process

In [10]:
# faiss 벡터 db에서 유사한 k개를 검색하여 내용을 리스트화함 
def retrieve_documents(query: str, faiss_index: FAISS, k: int = 3) -> List[str]:
    docs = faiss_index.similarity_search(query, k=k)
    return [doc.page_content for doc in docs]

# 쿼리와 문서 관련성 점수 출력 함수를 활용하여 모든 문서에 대한 점수 출력 
def evaluate_documents(query: str, documents: List[str]) -> List[float]:

    return [retrieval_evaluator(query, doc) for doc in documents]

# 검색 내용의 핵심 내용, 제목, 링크 추출 
def perform_web_search(query: str) -> Tuple[List[str], List[Tuple[str, str]]]:
    # 검색할 형식에 맞춰 쿼리를 재작성 한 후 검색 -> 검색 내용의 핵심 내용을 추출, 검색 내용의 제목과 링크 추출 
    rewritten_query = rewrite_query(query)
    web_results = search.run(rewritten_query)
    web_knowledge = knowledge_refinement(web_results)
    sources = parse_search_results(web_results)
    return web_knowledge, sources


# 검색한 내용을 바탕으로 쿼리에 대해 응답하는 함수 
def generate_response(query: str, knowledge: str, sources: List[Tuple[str, str]]) -> str:
    # 쿼리, 검색 내용의 핵심 내용, 제목, 링크를 넣어 해당 내용을 바탕으로 쿼리에 대한 응답을 생성
    # 정답의 마지막에 링크를 넣는 응답 
    response_prompt = PromptTemplate(
        input_variables=["query", "knowledge", "sources"],
        template="Based on the following knowledge, answer the query. Include the sources with their links (if available) at the end of your answer:\nQuery: {query}\nKnowledge: {knowledge}\nSources: {sources}\nAnswer:"
    )
    input_variables = {
        "query": query,
        "knowledge": knowledge,
        "sources": "\n".join([f"{title}: {link}" if link else title for title, link in sources])
    }
    response_chain = response_prompt | llm
    return response_chain.invoke(input_variables).content


### CRAG process


In [11]:
def crag_process(query: str, faiss_index: FAISS) -> str:
    
    # 처리할 쿼리 지명 
    print(f"\nProcessing query: {query}")

    # faiss에서 문서 반환 3개 
    retrieved_docs = retrieve_documents(query, faiss_index)
    # 관련도 점수 3개 
    eval_scores = evaluate_documents(query, retrieved_docs)
    
    print(f"\nRetrieved {len(retrieved_docs)} documents")
    print(f"Evaluation scores: {eval_scores}")

    # 최고 점수
    max_score = max(eval_scores)
    sources = []
    
    # 최고 점수가 0.7을 넘을 때 충분히 관련성이 높다고 생각하여 반환된 문서 그대로 사용 
    # 0.3보다 낮을 때 웹서칭
    # 그 사이면 반환된 문서의 중요한 정보만 뽑고, 검색한 문서의 중요한 정보만 뽑아서 둘을 합침. 
    if max_score > 0.7:
        print("\nAction: Correct - Using retrieved document")
        best_doc = retrieved_docs[eval_scores.index(max_score)]
        final_knowledge = best_doc
        sources.append(("Retrieved document", ""))
    elif max_score < 0.3:
        print("\nAction: Incorrect - Performing web search")
        final_knowledge, sources = perform_web_search(query)
    else:
        print("\nAction: Ambiguous - Combining retrieved document and web search")
        best_doc = retrieved_docs[eval_scores.index(max_score)]
        # Refine the retrieved knowledge
        retrieved_knowledge = knowledge_refinement(best_doc)
        web_knowledge, web_sources = perform_web_search(query)
        final_knowledge = "\n".join(retrieved_knowledge + web_knowledge)
        sources = [("Retrieved document", "")] + web_sources

# 최종 결과 반환 
    print("\nFinal knowledge:")
    print(final_knowledge)

# 검색한 링크와 제목 반환 
    print("\nSources:")
    for title, link in sources:
        print(f"{title}: {link}" if link else title)

# 최종 결과와 링크와 쿼리를 넣어 최종 응답 
    print("\nGenerating response...")
    response = generate_response(query, final_knowledge, sources)

    print("\nResponse generated")
    return response

### Example query with high relevance to the document


In [13]:
query = "What are the main causes of climate change?"
result = crag_process(query, vectorstore)


Processing query: What are the main causes of climate change?

Retrieved 3 documents
Evaluation scores: [0.9, 0.9, 0.6]

Action: Correct - Using retrieved document

Final knowledge:
driven by human activities, particularly the emission of greenhou se gases.  
Chapter 2: Causes of Climate Change  
Greenhouse Gases  
The primary cause of recent climate change is the increase in greenhouse gases in the 
atmosphere. Greenhouse gases, such as carbon dioxide (CO2), methane (CH4), and nitrous 
oxide (N2O), trap heat from the sun, creating a "greenhouse effect." This effect is  essential 
for life on Earth, as it keeps the planet warm enough to support life. However, human 
activities have intensified this natural process, leading to a warmer climate.  
Fossil Fuels  
Burning fossil fuels for energy releases large amounts of CO2. This includes coal, oil, and 
natural gas used for electricity, heating, and transportation. The industrial revolution marked 
the beginning of a significant increas

In [14]:
print(f"Query: {query}")
print(f"Answer: {result}")

Query: What are the main causes of climate change?
Answer: The main causes of climate change are primarily driven by human activities that lead to the emission of greenhouse gases. The key factors include:

1. **Greenhouse Gases**: The increase in greenhouse gases such as carbon dioxide (CO2), methane (CH4), and nitrous oxide (N2O) in the atmosphere is the primary cause of recent climate change. These gases trap heat from the sun, creating a "greenhouse effect" that is essential for maintaining life on Earth. However, human activities have intensified this natural process, resulting in a warmer climate.

2. **Burning of Fossil Fuels**: The combustion of fossil fuels (coal, oil, and natural gas) for energy is a significant contributor to CO2 emissions. This includes their use in electricity generation, heating, and transportation. The industrial revolution marked a substantial increase in fossil fuel consumption, which has continued to rise, exacerbating climate change.

3. **Deforestat

### Example query with low relevance to the document


In [ ]:
query = "how did harry beat quirrell?"
result = crag_process(query, vectorstore)
print(f"Query: {query}")
print(f"Answer: {result}")

## 내 생각
- 반환된 문서가 쿼리랑 관련이 없다고 생각했을 때 웹서칭을 통해 최신 관련 정보를 검색하는 것
- rag를 강화할 수 있는 큰 기술
- 검색 방법 : https://bcho.tistory.com/1428